In [330]:
import scipy.io
import numpy as np
import datetime 
import ephem
import random
from scipy import spatial
import matplotlib.pyplot as plt
%matplotlib
from astropy.table import Table
import astropy.units as u
import pandas as pd
from matplotlib.backends.backend_pdf import PdfPages
import astropy
from astropy.time import Time
from astropy.coordinates import EarthLocation
import sunpy.coordinates
from datetime import datetime, timezone
from math import atan, atan2, degrees, floor, pi, radians, sin, sqrt
from skyfield.api import earth, JulianDate, now, sun
data=scipy.io.loadmat('BNS-sources-trial-10saleem.mat')['data'] 
#sourceparameters
DL=data[:,0]
theta=data[:,1]
phi=data[:,2]

Using matplotlib backend: MacOSX


In [249]:
length = 600

In [540]:
def radectolatlong(Ra,Dec,time): #input in degrees
    ra = np.deg2rad(Ra)
    dec = np.deg2rad(Dec)
    time_sid = (time.sidereal_time('apparent','greenwich')).rad
    Lat = dec
    Long = ra - time_sid 
    
    if Long<0:
        Long+=360
    return Lat,Long #output in degrees

def latlongtoradec(Lat,Long,time): #input in radians
    #lat = np.deg2rad(Lat)
    #long = np.deg2rad(Long)
    time_sid = (time.sidereal_time('apparent','greenwich')).rad
    
    DEC = Lat
    RA = Long + time_sid
    ra = np.rad2deg(RA)            
    if ra>360:
        ra-=360               
    elif ra<0:
        ra+=360
    dec = np.rad2deg(DEC)
    return ra,dec #output in degree


def getxyz(theta,phi):
    #if n==1:
        #lat, long
    theta = np.pi/2 - theta
    x=np.sin(theta)*np.cos(phi)
    y=np.sin(theta)*np.sin(phi)
    z=np.cos(theta)
    return x,y,z
def getnorm(vec):
    norm = np.linalg.norm(vec)
    #print (norm)
    return vec/norm

In [450]:
df = pd.read_csv('/Users/jinaypatel/Desktop/Daksha/Events(triggers).csv')

In [451]:
time = df.sort_values(by=['Time in jd'])
t = time['Time in jd']
homotime = Time(t,format='jd')

In [452]:
sunloc = astropy.coordinates.get_sun(homotime)

In [499]:
sunRA = sunloc.ra.deg  #sun's RA in earth's frame in degrees
sunDEC = sunloc.dec.deg #sun's DEC in earth's frame in degrees
sunDIST = sunloc.distance.AU*1.496e11 #sun distance from earth in m

In [468]:
sunLat = np.empty(length,dtype=object)
sunLong = np.empty(length,dtype=object)
for i in range(length):
    x = radectolatlong(sunRA[i],sunDEC[i],homotime[i])
    sunLat[i] = x[0]
    sunLong[i] = x[1]

In [455]:
plt.plot(sunDEC[0:length],sunRA[0:length])

In [462]:
plt.plot(homotime[0:length].value,sunLong[0:length])
plt.xlabel("Time")
plt.ylabel("Sun's Longitude (degrees)")
plt.title("Sun Long for approx 1 day")
#plt.savefig("sunlong1day.pdf")

Text(0.5,1,'Sun Long for approx 1 day')

In [474]:
line0 = 'ASTROSAT' #Daksha
line1 = '1 40930U 15052A   22302.17850942  .00000772  00000-0  19439-4 0  9990'
line2 = '2 40930   5.9975   0.5159 0009292 339.1937  20.7998 14.76165238112787'
line3 = '2 40930   5.9975   0.5159 0009292 339.1937 200.7998 14.76165238112787'

# line0 = 'SWIFT'#BAT
# line1 = '1 28485U 04047A   22302.18405080  .00001301  00000-0  60140-4 0  9997'
# line2 = '2 28485  20.5572 277.8742 0011565 130.4507 229.6936 15.03911323708409'
# line0 = 'FGRST (GLAST)'#Fermi
# line1 = '1 33053U 08029A   22100.45970249  .00000511  00000-0  12538-4 0  9998'
# line2 = '2 33053  25.5819 140.4060 0012690 120.1881 239.9912 15.11069703542313'
astrosat1 = ephem.readtle(line0,line1,line2)
astrosat2 = ephem.readtle(line0,line1,line3)
Radius = 6400000 #radius of earth in m
alt1 = np.empty(length,dtype=object)
alt2 = np.empty(length,dtype=object)
sublat1 = np.empty(length,dtype = object)
sublong1 = np.empty(length,dtype = object)
sublat2 = np.empty(length,dtype = object)
sublong2 = np.empty(length,dtype = object)
for i in range(length):
    time = (homotime[i].iso)
    astrosat1.compute(time)
    astrosat2.compute(time)
    alt1[i] = astrosat1.elevation
    alt2[i] = astrosat2.elevation
    sublat1[i] = (astrosat1.sublat)          #sublat in radians
    sublong1[i] = (astrosat1.sublong)        #sublong in radian
    sublat2[i] = (astrosat2.sublat)          #sublat in radians

In [541]:
satunit1 = np.empty(length,dtype=object)
sununit = np.empty(length,dtype=object)
zaxis = np.empty(length,dtype=object)
sunsat = np.empty(length,dtype=object)
#yaxis = np.empty(length,dtype=object)
satvecold = np.empty(length,dtype=object)
for i in range(length-1):
    satunit1[i] = getxyz(sublat1[i],sublong1[i])
    sununit[i] = getxyz(sunLat[i],sunLong[i])    #earth to sun vector
    zaxis[i] = np.array([sununit[i][0]*sunDIST[i],sununit[i][1]*sunDIST[i],sununit[i][2]*sunDIST[i]])
    #sununitplus = getxyz(sunLat[i+1],sunLong[i+1])
    #xtemp = getnorm([sununitplus[0]-sununit[i][0],sununitplus[1]-sununit[i][1],sununitplus[1]-sununit[i][1]])
    #yaxis = np.array(getnorm(np.cross(zaxis[i],xtemp)))
    #xaxis = np.array(getnorm(np.cross(yaxis,zaxis[i])))
    #for satellite
    #time = str(homotime[i].iso)
    #astrosat1.compute(time)
    s = (getnorm(getxyz(sublat1[i],sublong1[i])))
    satvecold[i] = np.array([s[0]*(alt1[i]+Radius),s[1]*(alt1[i]+Radius),s[2]*(alt1[i]+Radius)])
    sunsat[i] = getnorm(satvecold[i] - zaxis[i])   #sun to satellite vector in earth's frame

In [542]:
sunsatx = np.empty(length,dtype=object)
sunsaty = np.empty(length,dtype=object)
sunsatz = np.empty(length,dtype=object)
norm = np.empty(length,dtype=object)
thetasat = np.empty(length,dtype=object)
phisat = np.empty(length,dtype=object)
latvec = np.empty(length,dtype=object)
lonvec = np.empty(length,dtype=object)
Ra = np.empty(length,dtype=object)
Dec = np.empty(length,dtype=object)
for i in range(length-1): 
    sunsatx[i] = sunsat[i][0]
    sunsaty[i] = sunsat[i][1]
    sunsatz[i] = sunsat[i][2]
    #norm[i] = (sunsatx[i]**2 + sunsaty[i]**2 + sunsatz[i]**2)**0.5
    thetasat[i] = np.arccos((sunsatz[i]))
    sunsatprojxynorm = (sunsatx[i]**2 + sunsaty[i]**2 )**0.5
    phisat[i] = np.arccos((sunsatx[i])/(sunsatprojxynorm))
    latvec[i] = np.pi/2 - thetasat[i]
    lonvec[i] = phisat[i]
    b = latlongtoradec(latvec[i],lonvec[i],homotime[i])
    Ra[i] = b[0]  # in deg
    Dec[i] = b[1]  # in deg

In [543]:
plt.plot(homotime.value[0:600],Ra[0:600])

In [524]:
import datetime